In [16]:
import pandas as pd

# Define base directory for saving files
base_dir = r"C:\Users\Christopher\OneDrive - Syracuse University\PythonSportAnalytics\Section_8\Final_Project\CSV_Files"

# Define the paths to your CSV files
file_1 = base_dir + "\\fbs_total_team_rushing_receiving_stats_by_season_2022_2024.csv"
file_2 = base_dir + "\\fbs_total_team_passing_stats_by_season_2022_2024.csv"
file_3 = base_dir + "\\fbs_teams_record_by_season_2022_2024.csv"

# Load the CSV files into pandas DataFrames
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)
df3 = pd.read_csv(file_3)

# Rename 'School' column in df3 to 'Team' for consistency
df3.rename(columns={'School': 'Team'}, inplace=True)

# Merge all the DataFrames on 'Team' and 'Year'
df1 = df1.rename(columns={'Year': 'Year'})
df2 = df2.rename(columns={'Year': 'Year'})
df3 = df3.rename(columns={'Year': 'Year'})

# Merge all DataFrames on both 'Team' and 'Year'
combined_df = pd.merge(df1, df2, on=['Team', 'Year'], how='outer')  # Merge df1 and df2 on 'Team' and 'Year'
combined_df = pd.merge(combined_df, df3, on=['Team', 'Year'], how='outer')  # Merge with df3

# *ADDED LATER* Drop the duplicate 'G' columns ('G_x' and 'G_y') and keep the first one
if 'G_x' in combined_df.columns and 'G_y' in combined_df.columns:
    combined_df['G'] = combined_df['G_x'].combine_first(combined_df['G_y'])
    combined_df.drop(['G_x', 'G_y'], axis=1, inplace=True)

# Drop the 'Year Link' column
if 'Year Link' in combined_df.columns:
    combined_df.drop(['Year Link'], axis=1, inplace=True)

# Rename "Rec" column to "Total Receptions"
if 'Rec' in combined_df.columns:
    combined_df.rename(columns={'Rec': 'Total Receptions'}, inplace=True)

# Reorder columns to put 'G' and then 'W', 'L', 'Pct' after 'G'
columns = ['Team', 'Year', 'G', 'W', 'L', 'Pct'] + [col for col in combined_df.columns if col not in ['Team', 'Year', 'G', 'W', 'L', 'Pct']]
combined_df = combined_df[columns]

# Sort the DataFrame by 'Team' and 'Year' in descending order of 'Year'
combined_df = combined_df.sort_values(by=['Team', 'Year'], ascending=[True, False])

# Save the combined DataFrame to a new CSV file
output_file = base_dir + "\\combined_fbs_stats_2022_2024.csv"
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV file saved to {output_file}")

# Preview the combined data
print(combined_df.head())


Combined CSV file saved to C:\Users\Christopher\OneDrive - Syracuse University\PythonSportAnalytics\Section_8\Final_Project\CSV_Files\combined_fbs_stats_2022_2024.csv
        Team  Year   G   W   L    Pct  Rushing Att  Rushing Yds  Rushing Y/A  \
2  Air Force  2024  12   5   7  0.417          664         2688          4.1   
1  Air Force  2023  13   9   4  0.692          702         3660          5.3   
0  Air Force  2022  13  10   3  0.769          776         4247          5.6   
5      Akron  2024  12   4   8  0.333          339         1204          3.6   
4      Akron  2023  12   2  10  0.167          339          972          2.9   

   Rushing TD  ...   Yds  TD  TD%  Int  Int%   Y/A   AY/A   Y/C    Y/G   Rate  
2          24  ...  1077   5  3.6   10   7.2   7.8   5.27  17.4   89.8  107.9  
1          36  ...  1105   7  6.7    6   5.7  10.5   9.29  20.1   85.0  151.4  
0          37  ...   916   8  9.3    2   2.3  10.7  11.47  22.3   70.5  163.2  
5          11  ...  2789  16  3.

In [17]:
# Check new file for any missing or duplicate data
combined_file = base_dir + "\\combined_fbs_stats_2022_2024.csv"

# Load the combined CSV file into a pandas DataFrame
combined_df = pd.read_csv(combined_file)

# Check if each unique team has one entry per season 2022, 2023, 2024
required_years = [2022, 2023, 2024]

# Group by 'Team' and count the number of unique years for each team
team_year_counts = combined_df.groupby('Team')['Year'].nunique()

# Check for teams that do not have exactly 3 unique seasons (2022, 2023, 2024)
teams_missing_years = team_year_counts[team_year_counts != 3]

# Print teams missing seasons
if teams_missing_years.empty:
    print("All teams have one entry per season (2022, 2023, 2024).")
else:
    print("The following teams are missing entries for one or more seasons:")
    print(teams_missing_years)

# Check for potential duplicates (same team and same year)
duplicates = combined_df[combined_df.duplicated(subset=['Team', 'Year'], keep=False)]

# Print duplicate rows if they exist
if not duplicates.empty:
    print("\nPotential duplicate entries found:")
    print(duplicates)
else:
    print("\nNo duplicate entries found.")

All teams have one entry per season (2022, 2023, 2024).

No duplicate entries found.
